In [ ]:
import sys
sys.path.insert(0,'..')
from data.preprocess import setup_path, ROOT_PATH

In [ ]:
from tqdm import tqdm
import os
def get_arg(textline, field):
    return eval(textline[textline.index(field+'='):textline.find(',',textline.index(field+'='),-1)].split('=')[1])

def extract_results(log_root_path, customized_args = [], file_name_identifier = "train_and_eval"):
    result_dict = {}
    for j,file in tqdm(enumerate(os.listdir(log_root_path))):
        if file_name_identifier in file:
            print(file)
            args = None
            model_name = ""
            results = []
            found = 0
            with open(os.path.join(log_root_path, file), 'r') as fin:
                for i,line in enumerate(fin):
                    if i == 0:
                        model_name = get_arg(line, 'model')
                    if i == 1:
                        args = line.strip()[10:-1]
                    elif "Test set performance" in line:
                        found = 2
                    elif found > 0:
                        found -= 1
                        if found == 0:
                            results.append(eval(line))
            if len(results) > 0:
                args += ','
                result_dict[j] = {'args': args}
                result_dict[j]['model_name'] = model_name
                for k in customized_args:
                    try:
                        result_dict[j][k] = get_arg(args, k)
                    except:
                        result_dict[j][k] = 'NaN'
                results = {k:[result[k] for result in results] for k in results[0].keys()}
                for k,v in results.items():
                    result_dict[j][k] = v
    return result_dict

## 1. Extract Results from Logs

In [ ]:
# from utils import extract_results, get_args
log_path = ROOT_PATH + "experiments/fedct/env_logs/"
control_args = ['lr', 'l2_coef', 'loss']
raw_results = extract_results(log_path, control_args, "fedct_train_and_eval")
print(len(raw_results))

In [ ]:
results = {}
for idx, R in raw_results.items():
    updated_results = {}
    for k,v in R.items():
        updated_results[k] = v
        if k == 'args':
            updated_results['domain'] = get_arg(R['args'], 'reader_path')[:-4].split('/')[-1][15:]
    results[idx] = updated_results
print(results.keys())

In [ ]:
import numpy as np
metrics = [f"{m_name}@{i}" for i in [1,5,10,20,50] for m_name in ['HR', 'RECALL', 'P', 'F1', 'NDCG']] + ["AUC", "MR", "MRR"]
for m_name in metrics:
    for k, res_dict in results.items():
        if m_name in res_dict:
            res_dict[m_name] = np.mean(res_dict[m_name])

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(results, orient = 'index')
df[:3]

In [ ]:
import os
import datetime
target_path = 'pretrain/'
setup_path(target_path, is_dir = True)
result_file_path = target_path + "env_models_performances.csv"
df.to_csv(result_file_path, sep = '\t')